In [1]:
import matplotlib
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure as Figure
import pylab
import sys
import os
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
from skimage.draw import polygon
import json
import matplotlib.image as mpimg
coco=COCO('') # add file 

In [2]:
def get_mask_polygons(anns, catIds=[]):
    polygons = []
    for ann in anns:
        if ann['category_id'] in catIds:
            # polygon
            for seg in ann['segmentation']:
                poly = np.array(seg).reshape((len(seg)//2, 2))
                polygons.append(Polygon(poly, True,alpha=0.4))
    return polygons    

def mask_objs(ann, imgId, catId, split):
        fig = Figure()
        ax = fig.gca()

        img_name = str(imgId).zfill(12)
        I=mpimg.imread('/scratch/network/nmeister/COCO/{0}/COCO_{0}_{1}.jpg' %(split, img_name))
        # load and display instance annotations
        plt.cla()
        plt.imshow(I); #plt.axis('off')
        annIds = coco.getAnnIds(imgIds=imgId, iscrowd=False)
        anns = coco.loadAnns(annIds)
        polygons = get_mask_polygons(anns, [catId])

        #code to make color average of scene
        colors = []
        if I.ndim == 2:
            I = np.tile(I[..., np.newaxis], (1, 1, 3))
        for channel in range(3):
            colors.append(np.mean(I[:,:,channel])/255.)
        p = PatchCollection(polygons, facecolors=tuple(colors), edgecolors=tuple(colors), linewidths=10)

        ax = plt.gca()
        ax.add_collection(p)
        ax.set_axis_off()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)


        ax.xaxis.set_major_locator(matplotlib.ticker.NullLocator())
        ax.yaxis.set_major_locator(matplotlib.ticker.NullLocator())

        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
                    hspace = 0, wspace = 0)

        save_dir = '/scratch/network/nmeister/COCO/{}/'.format(catId)
        if not os.path.isdir(save_dir):
            os.mkdir(save_dir)
        plt.savefig(save_dir + '{}.jpg'.format(img_name), bbox_inches='tight', pad_inches=-0.05)